In [29]:
import pandas as pd
import numpy as np

In [102]:
df = pd.read_csv("spam_classification/spam.csv",encoding="ISO-8859-1")
df = df[['v1','v2']]
df.columns = ['labels','message']
df

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, KeyedVectors
from nltk.stem import WordNetLemmatizer

In [4]:
# 1
corpus = []
flat_corpus = []
for doc in df['message']:
    corpus.append(simple_preprocess(doc))
    flat_corpus.extend(simple_preprocess(doc))

In [42]:
len(corpus)

5572

In [5]:
flat_corpus

['go',
 'until',
 'jurong',
 'point',
 'crazy',
 'available',
 'only',
 'in',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'there',
 'got',
 'amore',
 'wat',
 'ok',
 'lar',
 'joking',
 'wif',
 'oni',
 'free',
 'entry',
 'in',
 'wkly',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 'st',
 'may',
 'text',
 'fa',
 'to',
 'to',
 'receive',
 'entry',
 'question',
 'std',
 'txt',
 'rate',
 'apply',
 'over',
 'dun',
 'say',
 'so',
 'early',
 'hor',
 'already',
 'then',
 'say',
 'nah',
 'don',
 'think',
 'he',
 'goes',
 'to',
 'usf',
 'he',
 'lives',
 'around',
 'here',
 'though',
 'freemsg',
 'hey',
 'there',
 'darling',
 'it',
 'been',
 'week',
 'now',
 'and',
 'no',
 'word',
 'back',
 'like',
 'some',
 'fun',
 'you',
 'up',
 'for',
 'it',
 'still',
 'tb',
 'ok',
 'xxx',
 'std',
 'chgs',
 'to',
 'send',
 'to',
 'rcv',
 'even',
 'my',
 'brother',
 'is',
 'not',
 'like',
 'to',
 'speak',
 'with',
 'me',
 'they',
 'treat',
 'me',
 'like',
 'aids',
 'patent',
 'as',
 'pe

In [6]:
# 2
message = df.message.apply(simple_preprocess)
print(message[0])

['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']


In [41]:
message

0       [go, until, jurong, point, crazy, available, o...
1                             [ok, lar, joking, wif, oni]
2       [free, entry, in, wkly, comp, to, win, fa, cup...
3          [dun, say, so, early, hor, already, then, say]
4       [nah, don, think, he, goes, to, usf, he, lives...
                              ...                        
5567    [this, is, the, nd, time, we, have, tried, con...
5568           [will, ì_, going, to, esplanade, fr, home]
5569    [pity, was, in, mood, for, that, so, any, othe...
5570    [the, guy, did, some, bitching, but, acted, li...
5571                     [rofl, its, true, to, its, name]
Name: message, Length: 5572, dtype: object

In [7]:
# 3
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()
def preprocess_message(text):
    words = simple_preprocess(text, deacc=True)      # gensim processing remove punctuation,lowercase etc
    words = [word for word in words if word not in set(stopwords.words('english'))]    # removes stopwords (can be done in simple_preprocess)
    words = [lemmatizer.lemmatize(word) for word in words]    # get the words in there root form
    return words

In [14]:
processed_text = df.message.apply(preprocess_message)
processed_text

0       [go, jurong, point, crazy, available, bugis, g...
1                             [ok, lar, joking, wif, oni]
2       [free, entry, wkly, comp, win, fa, cup, final,...
3                    [dun, say, early, hor, already, say]
4             [nah, think, go, usf, life, around, though]
                              ...                        
5567    [nd, time, tried, contact, pound, prize, claim...
5568                     [i_, going, esplanade, fr, home]
5569                             [pity, mood, suggestion]
5570    [guy, bitching, acted, like, interested, buyin...
5571                                   [rofl, true, name]
Name: message, Length: 5572, dtype: object

In [17]:
processed_text[10]

['gonna',
 'home',
 'soon',
 'want',
 'talk',
 'stuff',
 'anymore',
 'tonight',
 'cried',
 'enough',
 'today']

In [ ]:
# Model Creation- word2vec
model = Word2Vec(processed_text, 
                window=5,
                min_count=3,
                workers=4,
                epochs=5,
                sg=0)

In [ ]:
# Build vocabulary of unique words in your corpus
model.build_vocab(processed_text,progress_per=1000)

In [21]:
# Model vocabulary
model.wv.index_to_key[:10]

['call', 'get', 'ur', 'gt', 'lt', 'go', 'ok', 'day', 'free', 'know']

In [11]:
# Vocab Count
model.corpus_count

5572

In [ ]:
# USing this before train would give irrelevant answers
model.wv.most_similar('call')

[('great', 0.9997209906578064),
 ('entitled', 0.9997090101242065),
 ('cup', 0.9997082352638245),
 ('early', 0.9997078776359558),
 ('send', 0.9997021555900574),
 ('even', 0.9996998906135559),
 ('la', 0.9996956586837769),
 ('apply', 0.9996935725212097),
 ('receive', 0.9996896386146545),
 ('comp', 0.9996885657310486)]

In [23]:
# Train model to get the Word embedding
model.train(processed_text, total_examples=model.corpus_count, epochs=model.epochs)

(190933, 238100)

In [24]:
model.wv.most_similar('call')

[('mobile', 0.9985458254814148),
 ('free', 0.9984306693077087),
 ('line', 0.9982403516769409),
 ('claim', 0.9981746673583984),
 ('reply', 0.9979180693626404),
 ('stop', 0.9979084730148315),
 ('service', 0.9978160858154297),
 ('award', 0.9977996349334717),
 ('land', 0.997785210609436),
 ('nokia', 0.9977737665176392)]

In [ ]:
# Cosine similarity
model.wv.similarity(w1='life',w2='home')

0.99918646

In [ ]:
model.wv.similarity(w1='love', w2="love")   # Same words points in same directions

1.0

In [34]:
processed_text

0       [go, jurong, point, crazy, available, bugis, g...
1                             [ok, lar, joking, wif, oni]
2       [free, entry, wkly, comp, win, fa, cup, final,...
3                    [dun, say, early, hor, already, say]
4             [nah, think, go, usf, life, around, though]
                              ...                        
5567    [nd, time, tried, contact, pound, prize, claim...
5568                     [i_, going, esplanade, fr, home]
5569                             [pity, mood, suggestion]
5570    [guy, bitching, acted, like, interested, buyin...
5571                                   [rofl, true, name]
Name: message, Length: 5572, dtype: object

In [69]:
# Average Word2Vec - instead of training on each word we average the vectors present in a sentence to create a single vector of vector_size(100 in this case) 
def average_wor2vec(doc):
    vectors = []
    for word in doc:
        if word in model.wv:
            vectors.append(model.wv[word])
    if len(vectors)>0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.wv.vector_size)

In [70]:
X = []
for i in range(len(corpus)):
    X.append(average_wor2vec(corpus[i]))
len(X)

5572

In [71]:
# Each sentence is of 100 vector_size ith total number of vectors to be equal to the total number of sentences in corpus
# we can train a model with this data and the label y 
X[0].shape

(100,)

In [80]:
X_new = np.array(X)
type(X_new)
X_new.shape, X_new[0].shape

((5572, 100), (100,))

In [ ]:
#Create dataset
data = pd.DataFrame()
for i in range(len(X_new)):
    row_df = pd.DataFrame(X_new[i].reshape(1,-1))
    data = pd.concat([data,row_df], ignore_index=True)

In [101]:
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.360381,0.726951,-0.004929,-0.000742,0.191843,-0.769369,0.228302,1.175514,-0.389787,-0.314993,...,0.691567,0.421490,-0.074873,0.052011,0.896856,0.480354,0.327865,-0.522430,0.397006,-0.015559
1,-0.331615,0.694344,-0.001217,-0.005753,0.179896,-0.739937,0.212811,1.124401,-0.374001,-0.299699,...,0.674987,0.407285,-0.083245,0.045788,0.842231,0.464324,0.308506,-0.496396,0.376604,-0.007080
2,-0.314709,0.676056,0.001604,-0.013217,0.182206,-0.713704,0.223071,1.094583,-0.358217,-0.291911,...,0.664804,0.409778,-0.075098,0.046739,0.818652,0.453722,0.291768,-0.494819,0.375700,-0.013179
3,-0.327303,0.682977,-0.000050,0.000764,0.173730,-0.740432,0.208635,1.123887,-0.372594,-0.294973,...,0.663004,0.407253,-0.077878,0.049849,0.848887,0.472967,0.301610,-0.491571,0.369320,-0.018274
4,-0.311799,0.634257,-0.015606,-0.011021,0.175903,-0.689086,0.195363,1.035223,-0.340030,-0.271396,...,0.623154,0.380831,-0.076513,0.042559,0.796125,0.444685,0.273677,-0.473254,0.342867,-0.015527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,-0.261219,0.574385,0.000558,-0.018441,0.158640,-0.584046,0.191182,0.896739,-0.279903,-0.241691,...,0.561508,0.348223,-0.057890,0.031744,0.672392,0.381286,0.248779,-0.420746,0.323453,-0.000190
5568,-0.194020,0.393461,-0.003060,0.003141,0.099436,-0.440317,0.121568,0.661173,-0.219591,-0.171852,...,0.392704,0.234027,-0.057018,0.025564,0.497509,0.270841,0.178778,-0.293671,0.222291,-0.011789
5569,-0.065337,0.154372,-0.011158,-0.000515,0.039768,-0.155871,0.035389,0.232453,-0.089420,-0.052491,...,0.150891,0.084557,-0.025391,0.007389,0.184541,0.103067,0.058244,-0.104517,0.072802,-0.011400
5570,-0.235653,0.484030,-0.001654,-0.010313,0.129689,-0.517594,0.158449,0.800803,-0.263721,-0.206909,...,0.476128,0.298117,-0.050702,0.038528,0.596378,0.328545,0.215080,-0.353060,0.267782,-0.007459


In [103]:
# Dependent feature
y = pd.get_dummies(df['labels'])
y = y.iloc[:, 0].values

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,y, test_size=0.20)

In [105]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()

In [106]:
classifier.fit(X_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [107]:
y_pred = classifier.predict(X_test)

In [110]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred,y_test))

0.968609865470852
